<a href="https://colab.research.google.com/github/saurater/machine_learning/blob/main/SKlearn_XGBoost_Otimiza%C3%A7%C3%A3o_com_Hiperpar%C3%A2metros_Kaggle_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sklearn - XGboost - Otimização com Hiperparâmetros - Kaggle - V1

**Sam Faraday**

#  Importando Bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lendo Datasets

In [2]:
X           = pd.read_csv('train.csv', index_col='Id')
# https://github.com/saurater/machine_learning/blob/main/test.csv

X_test_full = pd.read_csv('test.csv', index_col='Id')
# https://github.com/saurater/machine_learning/blob/main/train.csv

# Removendo linhas sem o Alvo (SalePrice) e Separando o Alvo das Preditoras

In [3]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

y = X.SalePrice            
  
X.drop(['SalePrice'], axis=1, inplace=True)

# Separando Dados de Treino e Validação

In [4]:
X_train_full, X_valid_full, y_train, y_valid = \
train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Cardinalidade - Selecionando Colunas Categóricas 
# com menos de 10 Valores Únicos

In [5]:
colunas_com_baixa_cardinalidade = \
[cname for cname in X_train_full.columns \
 if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Selecionando Colunas Numéricas

In [6]:
colunas_numericas = [cname for cname in X_train_full.columns\
                if X_train_full[cname].dtype in ['int64', 'float64']]

# Unindo Colunas Categóricas e Numéricas

In [7]:
minhas_colunas = colunas_com_baixa_cardinalidade + colunas_numericas

X_train = X_train_full[minhas_colunas].copy()
X_valid = X_valid_full[minhas_colunas].copy()
X_test = X_test_full[minhas_colunas].copy()

# One Hot Encoding com Pandas

Note que temos 76 Features. Depois do Get Dummy Encoding esse número passa para 227

In [8]:
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test  = pd.get_dummies(X_test)

# ATENÇÃO: Alinhando os DataFrames - Necessário com Get Dummies

 Ao se dividir o dataset em mais de um (Treino, Validação e Testes), pode ser que um determinado dataset tenha categorias de uma determinada feature que não estão presentes no outro dataset, tipo de testes e validação.

 Sem fazer o Alinhamento de Dataset, eles estarão desbalanceados e seu modelo não terá a perfeição mínima desejada.

 Por exemplo: imagina que tenha uma feature de Color com 3 variações. No DF_Treino podem ir as 3 cores, mas no de Teste podem ir apenas 1 ou 2 Cores, e sabemos que o Get Dummies vai criar tantas colunas quantas forem as categorias de cada feature (coluna)

In [9]:
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test  = X_train.align(X_test, join='left', axis=1)

# 1o. Modelo

## Criando o Primeiro Modelo - Hiperparâmetros Padrões


In [10]:
from xgboost import XGBRegressor

# Define the model
meu_modelo_1 = XGBRegressor(random_state=0, verbosity = 0)

# Fit the model
meu_modelo_1.fit(X_train, y_train)


XGBRegressor(verbosity=0)

## 1a. Previsão

In [11]:
from sklearn.metrics import mean_absolute_error

previsao_1 = meu_modelo_1.predict(X_valid)

previsao_1 = pd.DataFrame(previsao_1, columns=['Previsao 1'])

previsao_1 

,Previsao 1
0,237696.218750
1,159064.046875
2,108627.734375
3,212797.796875
4,88389.437500
...,...
287,287674.625000
288,551785.687500
289,158480.593750
290,79905.507812


## 1o. MAE - Mean Absolute Error
Erro Abosluto Médio - Quanto Menor, Melhor

In [12]:
mae_1 = round(mean_absolute_error(previsao_1, y_valid))
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 16803


# 2o. Modelo - n_estimators=1000, learning_rate=0.05

In [13]:
# Define the model
meu_modelo_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05, verbosity = 0)

# Fit the model
meu_modelo_2.fit(X_train, y_train)

XGBRegressor(learning_rate=0.05, n_estimators=1000, verbosity=0)

## 2a. Previsão

In [14]:
previsao_2 = meu_modelo_2.predict(X_valid)

## 2o. MAE - Mean Absolute Error
Erro Abosluto Médio - Quanto Menor, Melhor

In [15]:
# Calcula MAE
mae_2 = round(mean_absolute_error(previsao_2, y_valid))
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 16084


# 3o. Modelo com apenas uma Árvore (n_estimators=1)
Espera-se uma MAE maior, ou seja, um modelo pior

In [16]:
meu_modelo_3 = XGBRegressor(n_estimators=1, verbosity=0)

# Ajusta o Modelo
meu_modelo_3.fit(X_train, y_train)

XGBRegressor(n_estimators=1, verbosity=0)

## 3a. Previsão

In [17]:
previsao_3 = meu_modelo_3.predict(X_valid)

## 3o. MAE - Mean Absolute Error
Erro Abosluto Médio - Quanto Menor, Melhor

In [18]:
mae_3 = round(mean_absolute_error(previsao_3, y_valid))
print("Mean Absolute Error:" , mae_3)

Mean Absolute Error: 163383


# Comparando os Modelos

In [19]:
dic_comparacao1 = {"MAE": mae_1, "n_estimators": "padrão","learning_rate": "padrão"}

dic_comparacao2 = {"MAE": mae_2, "n_estimators": "1000","learning_rate": "0.5"}

dic_comparacao3 = {"MAE": mae_3, "n_estimators": "1","learning_rate": "padrao"}

In [20]:
df_comparacao = pd.DataFrame([dic_comparacao1, dic_comparacao2, dic_comparacao3], index=["1a", "2a", "3a"])

df_comparacao= df_comparacao.sort_values("MAE")

df_comparacao

,MAE,n_estimators,learning_rate
2a,16084,1000,0.5
1a,16803,padrão,padrão
3a,163383,1,padrao
